# 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

In [1]:
# Loading library
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# Setting hyper-parameters
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [3]:
# Check data formation
(x_train_orginal, y_train_orginal), (x_test_orginal, y_test_orginal) = cifar10.load_data()
print('x_train shape:', x_train_orginal.shape)
print('x_test shape:', x_test_orginal.shape)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)


In [4]:
x_train = x_train_orginal
x_test = x_test_orginal

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train_orginal, num_classes)
y_test = to_categorical(y_test_orginal, num_classes)

In [12]:
class Model():
    def __init__(self, x_train, y_train, x_test, y_test):
        self.FirConvSize = None
        self.SecConvSize = None
        self.Padding = None
        self.model = None
        self.KernlSize = None
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
    
    def setHypParam(self, padding='same', KernlSize=(3, 3), FirConvSize=32, SecConvSize=64):
        self.FirConvSize = FirConvSize
        self.SecConvSize = SecConvSize
        self.KernlSize = KernlSize
        self.Padding = padding
    
    def buildModel(self):
        self.model = Sequential()
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize, padding=self.Padding,
                         input_shape=x_train.shape[1:]))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.FirConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize, padding=self.Padding))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(self.SecConvSize, self.KernlSize))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Dropout(0.25))

        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Dense(num_classes))
        self.model.add(Activation('softmax'))
    
    def setCompile(self):
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(),
                           metrics=['accuracy'])
    
    def fitModel(self):
        self.model.fit(self.x_train, self.y_train,
                       batch_size=128, 
                       epochs=10,
                       verbose=False,
                       validation_data=(self.x_test, self.y_test))
    
    def evaluateScore(self):
        return (self.model.evaluate(self.x_test, self.y_test, verbose=0))

In [14]:
# Discissing effection of conv size.
# setting hyper-parameters
FirConvSize_list = [16, 32, 64]
SecConvSize_list = [32, 64, 128]

for FirConvSize in FirConvSize_list:
    for SecConvSize in SecConvSize_list:
        print("Fir_Conv_size:%d, Sec_Conv_size:%d" %(FirConvSize, SecConvSize))
        model = Model(x_train, y_train, x_test, y_test)
        model.setHypParam(FirConvSize=FirConvSize, SecConvSize=SecConvSize)
        model.buildModel()
        model.setCompile()
        model.fitModel()
        score = model.evaluateScore()
        print('Test loss:%.4f' %score[0])
        print('Test accuracy:%.4f' %score[1])
        print("")

Fir_Conv_size:16, Sec_Conv_size:32
Test loss:0.7868
Test accuracy:0.7293

Fir_Conv_size:16, Sec_Conv_size:64
Test loss:0.8198
Test accuracy:0.7288

Fir_Conv_size:16, Sec_Conv_size:128
Test loss:0.7601
Test accuracy:0.7666

Fir_Conv_size:32, Sec_Conv_size:32
Test loss:0.7570
Test accuracy:0.7385

Fir_Conv_size:32, Sec_Conv_size:64
Test loss:0.7258
Test accuracy:0.7575

Fir_Conv_size:32, Sec_Conv_size:128
Test loss:0.6964
Test accuracy:0.7764

Fir_Conv_size:64, Sec_Conv_size:32
Test loss:0.7860
Test accuracy:0.7339

Fir_Conv_size:64, Sec_Conv_size:64
Test loss:0.6995
Test accuracy:0.7766

Fir_Conv_size:64, Sec_Conv_size:128
Test loss:0.8113
Test accuracy:0.7620



# Discussion - 1
* 從上面的比較分析，調整各層的 filter 個數均可以有效的提升準確度，且趨勢為 filter 越多準度越高。
* 另外，可以發現越靠近輸出層的 filter 對於預測精度影響越大，因此可試著將靠近 output layer 的 filter 調高以求更高的準確度。

In [15]:
FirConvSize = 64
SecConvSize = 64
padding_list = ['same', 'VALID']

for padding in padding_list:
    print("Padding:%s, Fir_Conv_size:%d, Sec_Conv_size:%d" %(padding, FirConvSize, SecConvSize))
    model = Model(x_train, y_train, x_test, y_test)
    model.setHypParam(padding=padding, FirConvSize=FirConvSize, SecConvSize=SecConvSize)
    model.buildModel()
    model.setCompile()
    model.fitModel()
    score = model.evaluateScore()
    print('Test loss:%.4f' %score[0])
    print('Test accuracy:%.4f' %score[1])
    print("")

Padding:same, Fir_Conv_size:64, Sec_Conv_size:64
Test loss:0.6980
Test accuracy:0.7715

Padding:VALID, Fir_Conv_size:64, Sec_Conv_size:64
Test loss:0.7069
Test accuracy:0.7619



# Discussion - 2
* 使用 padding = "same" 可以得到更好的效果。
* 因為不會降低影像的維度，進而得到的資訊較多，因此預測效果較好。

In [19]:
padding = 'same'
KernelSize_list = [(3, 3), (5, 5), (8, 8)]

for KernelSize in KernelSize_list:
    print(KernelSize)
    print("Padding:%s, Fir_Conv_size:%d, Sec_Conv_size:%d" %(padding, FirConvSize, SecConvSize))
    model = Model(x_train, y_train, x_test, y_test)
    model.setHypParam(padding=padding, KernlSize=KernelSize, FirConvSize=FirConvSize, SecConvSize=SecConvSize)
    model.buildModel()
    model.setCompile()
    model.fitModel()
    score = model.evaluateScore()
    print('Test loss:%.4f' %score[0])
    print('Test accuracy:%.4f' %score[1])
    print("")

(3, 3)
Padding:same, Fir_Conv_size:64, Sec_Conv_size:64
Test loss:0.6480
Test accuracy:0.7783

(5, 5)
Padding:same, Fir_Conv_size:64, Sec_Conv_size:64
Test loss:0.7560
Test accuracy:0.7568

(8, 8)
Padding:same, Fir_Conv_size:64, Sec_Conv_size:64
Test loss:1.1538
Test accuracy:0.6003



# Discussion - 3
* 調整 Kernel 越小對於預測準度的提升越有幫助。

# Conclusion
* 對於結果有明顯影響的超參數只有 Kernel Size，剩下的超參數對於結果的影響差異並不大。